In [1]:
#import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [2]:
#import dataset
tele_df= pd.read_csv('bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


Eventhough SVM can handle tabular data. Process the data for both models to ensure fairo comparison

In [3]:
#ONE HOT ENCODER
#generate categorical variable list
tele_cat= tele_df.dtypes[tele_df.dtypes=='object'].index.tolist()

#check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [5]:
#create one hot encoder instance
enc= OneHotEncoder(sparse=False)

#fit and transform onehotencoder using categorical variable list
encode_df= pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

#add the encoded variable names to the dataframe
encode_df.columns= enc.get_feature_names(tele_cat)
encode_df.head()

,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [6]:
#merge encoded features and drop originals
tele_df= tele_df.merge(encode_df, left_index=True, right_index=True)
tele_df= tele_df.drop(tele_cat, 1)
tele_df.head()


,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


After cleaning the data. 

find features and target

Split into training and testing sets

standardize numerical variables

In [8]:
#remove loan status target from features data
y= tele_df.Subscribed_yes.values
X= tele_df.drop(columns=['Subscribed_yes', 'Subscribed_no']).values

#split into train/test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42, stratify=y)

#create standard scaler instance
scaler= StandardScaler()

#fit the standard scaler
X_scaler= scaler.fit(X_train)

#scale the data
X_train_scaled= X_scaler.transform(X_train)
X_test_scaled= X_scaler.transform(X_test)

Dataset is now ready for both models after standardization

In [9]:
#create SVM model
svm= SVC(kernel='linear')

#train model
svm.fit(X_train, y_train)

#evaluate the model
y_pred= svm.predict(X_test_scaled)
print(f'SVM model accuracy: {accuracy_score(y_test, y_pred):.3f}')

SVM model accuracy: 0.873


In [11]:
#define the model -deep neural net
number_input_features= len(X_train_scaled[0])
hidden_nodes_layer1= 10
hidden_nodes_layer2= 5

nn= tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer1, input_dim= number_input_features, activation='relu'))

#second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

#output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#compile the sequential model and customize metrics
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=50)

#evaluate model using testing data
model_loss, model_accuracy= nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'loss: {model_loss}, accuracy: {model_accuracy}')

Epoch 1/50
715/715 [==============================] - 1s 765us/step - loss: 0.4854 - accuracy: 0.8119
Epoch 2/50
715/715 [==============================] - 0s 685us/step - loss: 0.3772 - accuracy: 0.8732
Epoch 3/50
715/715 [==============================] - 0s 676us/step - loss: 0.3731 - accuracy: 0.8719
Epoch 4/50
715/715 [==============================] - 0s 636us/step - loss: 0.3681 - accuracy: 0.8736
Epoch 5/50
715/715 [==============================] - 1s 762us/step - loss: 0.3639 - accuracy: 0.8762
Epoch 6/50
715/715 [==============================] - 1s 749us/step - loss: 0.3640 - accuracy: 0.8751
Epoch 7/50
715/715 [==============================] - 0s 612us/step - loss: 0.3609 - accuracy: 0.8776
Epoch 8/50
715/715 [==============================] - 0s 675us/step - loss: 0.3662 - accuracy: 0.8741
Epoch 9/50
715/715 [==============================] - 0s 655us/step - loss: 0.3696 - accuracy: 0.8719
Epoch 10/50
715/715 [==============================] - 0s 667us/step - loss: 0.371